In [1]:
import os
import sys
sys.path.append(os.path.abspath('../..'))
from nowoe.data.api.smartwatch.utilities.preparations import *
import pandas
import plotly_express as px
import numpy
import copy
import json
import pickle
import gzip
import matplotlib.pyplot as plt
from copy import deepcopy
from functools import reduce
from tqdm import tqdm
from datetime import datetime, timezone
from typing import Dict, List, Tuple, Union, Any, Iterator
import torch.utils.data.dataloader
from nowoe.data.api.smartwatch.utilities.timestamp import get_utc_date_from_utc_timestamp
from nowoe.data.api.smartwatch.data_manager.module import SmartwatchDataManager
from nowoe.deep_learning.data.dataset.smartwatch_study.single_slice import get_dataloaders, SmartwatchStudySingleSliceDataset, single_slice_collate_fn
from nowoe.deep_learning.data.augmentation.single_slice.sample_from_distribution.gaussian_mixtures import GaussianMixturesSingleSliceAugmentation

In [2]:
dataloaders = get_dataloaders(
    batch_size=50,
    root_dir='../../resources/warrior_wellness/Analysis/local_repo/',
    subject_splits={
        "train": [f'SWS_{i:02d}' for i in range(0,10)],
        "test": [f'SWS_{i:02d}' for i in range(10,15)]},
    dataset_config={
        'slice_lengths': [3600],
        'slice_time_step': (5 * 60),
        'label_milestone_per_window': 1.0,
        'metadata_cache_filepath': './dataset_cache/dataset-cache-2.pkl.gz',
        'no_cache': False,
        'parallel_threads': 10
    },
    sampler_configs=dict(
       train=dict(
           negative_sample_count=1000,
            positive_sample_count=500,
            target_variable='overall_quantized_stress_value',
           split_name="train"
       ),
       test=dict(
               negative_sample_count=200,
            positive_sample_count=100,
            target_variable='overall_quantized_stress_value',
           split_name="test"
       )
        )
)

2022-02-23 10:38:04,032 - nowoe.deep_learning.data.dataset.smartwatch_study.single_slice.interface - INFO - initializing data manager...
2022-02-23 10:40:38,202 - nowoe.deep_learning.data.dataset.smartwatch_study.single_slice.interface - INFO - preparing the dataset...
2022-02-23 10:40:38,508 - nowoe.deep_learning.data.dataset.smartwatch_study.single_slice.dataset - INFO - Loaded metadata from cache file: /home/shayan/phoenix/nowoe_framework/nowoe_framework/notebooks/smartwatch/dataset_cache/dataset-cache-2.pkl.gz
2022-02-23 10:40:38,509 - nowoe.deep_learning.data.dataset.smartwatch_study.single_slice.interface - INFO - preparing samplers...
2022-02-23 10:40:38,681 - nowoe.deep_learning.data.dataset.smartwatch_study.single_slice.sampler - WARNING - no samples for label 2.057142857142857 was found in the split: test
2022-02-23 10:40:38,682 - nowoe.deep_learning.data.dataset.smartwatch_study.single_slice.sampler - WARNING - no samples for label 2.314285714285714 was found in the split: t

In [3]:
augmentation = GaussianMixturesSingleSliceAugmentation(
feature_names_per_data_source={'daily': ['heart_rate_tsvalue']},
    gmm_config=dict(
    n_components=2,
    covariance_type='full'
))

In [4]:
augmentation.learn(dataloader=dataloaders['train'])

2022-02-23 10:40:38,695 - nowoe.deep_learning.data.augmentation.single_slice.sample_from_distribution.base - INFO - [learning distributions] sampling from the given dataloader...
100%|█████████████████████████████████████████| 160/160 [01:05<00:00,  2.45it/s]
2022-02-23 10:41:43,885 - nowoe.deep_learning.data.augmentation.single_slice.sample_from_distribution.base - INFO - [learning distributions] fitting models (of class <class 'sklearn.mixture._gaussian_mixture.GaussianMixture'>) to the distributions...
2022-02-23 10:41:45,962 - nowoe.deep_learning.data.augmentation.single_slice.sample_from_distribution.base - INFO - [learning distributions] completed.


In [5]:
my_iter = iter(dataloaders['test'])

In [6]:
batch = next(my_iter)

In [8]:
augmented_batch = augmentation.augment(copy.deepcopy(batch), rows_to_add=10)

In [11]:
batch['slice'][0]['daily'].shape

(240, 34)

In [12]:
augmented_batch['slice'][0]['daily'].shape

(250, 34)

In [13]:
new_timestamps = set(augmented_batch['slice'][0]['daily'].utc_timestamp.tolist()).difference(set(batch['slice'][0]['daily'].utc_timestamp.tolist()))

In [18]:
df = augmented_batch['slice'][0]['daily'].copy()

In [19]:
df[df.utc_timestamp.isin(new_timestamps)]

,utc_timestamp,heart_rate_tsvalue,restingHeartRateInBeatsPerMinute,lowStressDurationInSeconds,moderateIntensityDurationInSeconds,stepsGoal,durationInSeconds,summaryId,activityStressDurationInSeconds,floorsClimbedGoal,...,restStressDurationInSeconds,maxStressLevel,activeTimeInSeconds,startTimeOffsetInSeconds,steps,minHeartRateInBeatsPerMinute,highStressDurationInSeconds,user_id,bmrKilocalories,utc_date
241,1.626094e+09,101.197248,44.0,13380.0,0.0,9630.0,86400.0,x41fb4e4-60eb69e0-15180-6,23460.0,10.0,...,37320.0,87.0,4042.0,7200.0,9186.0,42.0,540.0,SWS_11,2117.0,2021-07-12 12:47:30+00:00
242,1.626094e+09,96.237144,44.0,13380.0,0.0,9630.0,86400.0,x41fb4e4-60eb69e0-15180-6,23460.0,10.0,...,37320.0,87.0,4042.0,7200.0,9186.0,42.0,540.0,SWS_11,2117.0,2021-07-12 12:50:00+00:00
240,1.626094e+09,78.547934,44.0,13380.0,0.0,9630.0,86400.0,x41fb4e4-60eb69e0-15180-6,23460.0,10.0,...,37320.0,87.0,4042.0,7200.0,9186.0,42.0,540.0,SWS_11,2117.0,2021-07-12 12:52:45+00:00
245,1.626095e+09,72.591211,44.0,13380.0,0.0,9630.0,86400.0,x41fb4e4-60eb69e0-15180-6,23460.0,10.0,...,37320.0,87.0,4042.0,7200.0,9186.0,42.0,540.0,SWS_11,2117.0,2021-07-12 12:58:15+00:00
243,1.626095e+09,85.157863,44.0,13380.0,0.0,9630.0,86400.0,x41fb4e4-60eb69e0-15180-6,23460.0,10.0,...,37320.0,87.0,4042.0,7200.0,9186.0,42.0,540.0,SWS_11,2117.0,2021-07-12 13:02:00+00:00
248,1.626096e+09,73.354650,44.0,13380.0,0.0,9630.0,86400.0,x41fb4e4-60eb69e0-15180-6,23460.0,10.0,...,37320.0,87.0,4042.0,7200.0,9186.0,42.0,540.0,SWS_11,2117.0,2021-07-12 13:11:45+00:00
247,1.626096e+09,63.138076,44.0,13380.0,0.0,9630.0,86400.0,x41fb4e4-60eb69e0-15180-6,23460.0,10.0,...,37320.0,87.0,4042.0,7200.0,9186.0,42.0,540.0,SWS_11,2117.0,2021-07-12 13:13:30+00:00
249,1.626096e+09,69.111677,44.0,13380.0,0.0,9630.0,86400.0,x41fb4e4-60eb69e0-15180-6,23460.0,10.0,...,37320.0,87.0,4042.0,7200.0,9186.0,42.0,540.0,SWS_11,2117.0,2021-07-12 13:20:30+00:00
246,1.626097e+09,74.773022,44.0,13380.0,0.0,9630.0,86400.0,x41fb4e4-60eb69e0-15180-6,23460.0,10.0,...,37320.0,87.0,4042.0,7200.0,9186.0,42.0,540.0,SWS_11,2117.0,2021-07-12 13:32:30+00:00
244,1.626097e+09,66.826390,44.0,13380.0,0.0,9630.0,86400.0,x41fb4e4-60eb69e0-15180-6,23460.0,10.0,...,37320.0,87.0,4042.0,7200.0,9186.0,42.0,540.0,SWS_11,2117.0,2021-07-12 13:37:45+00:00
